<a href="https://colab.research.google.com/github/Helerik/animal-movemet-models/blob/main/g1_models_build_train_test_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler as std_scaler, OneHotEncoder as OHE
import pickle
import json
import matplotlib.pyplot as plt
import time
import gc

print("\nPackages imported successfully\n")

# Handling jaguar movement data
class JaguarDataHandler():
    def __init__(self, directory = "/content/drive/MyDrive/Trabalho de Formatura/00 - Animal Movement/animal-movement-model/data/jaguar_data/"):
        self.directory = directory
        self.datasets = []
        self.training_Xs = []
        self.training_Ys = []
        
    def load_data(self, IDs = None):
        for id in IDs:
            name = "jag_"+str(id)+".csv"
            chunk = pd.read_csv(self.directory+name, sep = ",", decimal = ".", chunksize = 10000)
            data = pd.concat(chunk, ignore_index = True)
            del chunk
            self.datasets.append(
                data
            )
        return 0
    
    def data2training(self, variables, target_variables, target, step_ID, jaguar_ID = None,
                      del_IDs = True, del_original = False):
        
        if not target in target_variables:
            return -1
        if del_IDs and not step_ID:
            return -1
        
        for data in self.datasets:
            J = data.copy()
            X = J.filter(variables)
            Y = J.filter(target_variables)
            step_id = J[step_ID].unique()
            X_tf = []
            Y_tf = []
            for i in step_id:
                X_tmp = X[(X[step_ID] == i)]
                Y_tmp = Y[(Y[step_ID] == i)]
                Y_tmp = Y_tmp.filter([target])
                
                if del_IDs:
                    X_tmp.drop([jaguar_ID, step_ID], axis = 1, inplace = True)
                
                X_tf.append(X_tmp.to_numpy())
                Y_tf.append(Y_tmp.to_numpy().T)
            X_tf = tf.constant(X_tf)
            Y_tf = tf.constant(Y_tf)
            Y_tf = tf.reshape(Y_tf, [Y_tf.shape[0],Y_tf.shape[2]])
            self.training_Xs.append(X_tf)
            self.training_Ys.append(Y_tf)
        
        if del_original:
            self.datasets = []
            
        return 0

# Create data handler
data_handler = JaguarDataHandler()

# Importing Jaguars data
data_handler.load_data(IDs = [12,13,18,22,23,41,52,81,88,91,92,116,117])

variables = ['jaguar_ID', 'step_Num', 'step_length','hour','slope','topo','distdrain','distwater','tcover',
             'MaxNDVI', 'MedNDVI', 'landcover', 'humanfoot', 'popden', 'livestock', 'dist12', 'dist345', 'distanthropic',
             'distPA', 'month_Apr', 'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan', 'month_Jul', 'month_Jun',
             'month_Mar', 'month_May', 'month_Nov', 'month_Oct', 'month_Sep', 'landC_anthropic', 'landC_natural',
             'landC_water', 'landC2_agro', 'landC2_nat_cover', 'landC2_nat_uncover', 'landC3_forest', 'landC3_grassland',
             'landC3_pasture_mosaic', 'landC3_shrubland', 'landC3_wetland_mangrove', 'suggested_x', 'suggested_y']
target_variables = ['jaguar_ID', 'step_Num', 'case_']
target = 'case_'
step_ID = 'step_Num'
jaguar_ID = 'jaguar_ID'

data_handler.data2training(variables,target_variables,target,step_ID,jaguar_ID)


Packages imported successfully



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


0

In [4]:
# Create n_folds folds
n_folds = 7

x_train = tf.reverse(tf.concat(data_handler.training_Xs,axis=0),axis=[1])
y_train = tf.reverse(tf.concat(data_handler.training_Ys,axis=0),axis=[1])

indices = tf.range(start=0, limit=tf.shape(x_train)[0], dtype=tf.int32)
shuffled_indices = tf.random.shuffle(indices)

x_train = tf.gather(x_train, shuffled_indices)
y_train = tf.gather(y_train, shuffled_indices)

x_folds = tf.split(x_train,n_folds,axis=0)
y_folds = tf.split(y_train,n_folds,axis=0)

In [6]:
x_train.shape

TensorShape([9723, 201, 42])

In [ ]:
class noiseLayer(tf.keras.layers.Layer):

    def __init__(self,minVal,maxVal,seed = None):
        super(noiseLayer, self).__init__()
        self.minVal = minVal
        self.maxVal = maxVal
        self.seed = seed

    def call(self, inp):

        minVal = self.minVal
        maxVal  = self.maxVal
        seed = self.seed

        return inp + tf.random.uniform(tf.shape(inp), 
                                    minval = minVal,
                                    maxval = maxVal,
                                    seed = seed)

# Conditional Logistic Regression Model using Keras and Tensorflow APIs (with l1 and l2 regularization options)
class ConditionalLogisticRegression(tf.keras.Model):
  def __init__(self, out_dim, l1 = 0, l2 = 0):
    super(ConditionalLogisticRegression, self).__init__(name='ConditionalLogisticRegression')
    self.layer1 = tf.keras.layers.Dense(1,'linear',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2),)
    self.layer2 = tf.keras.layers.Softmax(axis=-2)
    self.noise = noiseLayer(0.00001,0.00009)
  def call(self, input_tensor, training=False):
    x = self.layer1(input_tensor,training=training)
    x = self.layer2(x,training=training)
    x = self.noise(x)
    return x

# Conditional NN Models using Keras and Tensorflow APIs (with l1 and l2 regularization options)
class ConditionalNN1(tf.keras.Model):
  def __init__(self, out_dim, l1 = 0, l2 = 0):
    super(ConditionalNN1, self).__init__(name='ConditionalNN1')
    self.layer1 = tf.keras.layers.Dense(10,'tanh',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL1 = tf.keras.layers.Dense(1,'linear',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL2 = tf.keras.layers.Softmax(axis=-2)
    self.noise = noiseLayer(0.00001,0.00009)
  def call(self, input_tensor, training=False):
    x = self.layer1(input_tensor,training=training)
    x = self.layerL1(x,training=training)
    x = self.layerL2(x,training=training)
    x = self.noise(x)
    return x

class ConditionalNN2(tf.keras.Model):
  def __init__(self, out_dim, l1 = 0, l2 = 0):
    super(ConditionalNN2, self).__init__(name='ConditionalNN2')
    self.layer1 = tf.keras.layers.Dense(20,'tanh',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL1 = tf.keras.layers.Dense(1,'linear',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL2 = tf.keras.layers.Softmax(axis=-2)
    self.noise = noiseLayer(0.00001,0.00009)
  def call(self, input_tensor, training=False):
    x = self.layer1(input_tensor,training=training)
    x = self.layerL1(x,training=training)
    x = self.layerL2(x,training=training)
    x = self.noise(x)
    return x

class ConditionalNN11(tf.keras.Model):
  def __init__(self, out_dim, l1 = 0, l2 = 0):
    super(ConditionalNN11, self).__init__(name='ConditionalNN11')
    self.layer1 = tf.keras.layers.Dense(10,'tanh',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer2 = tf.keras.layers.Dense(10,'tanh',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL1 = tf.keras.layers.Dense(1,'linear',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL2 = tf.keras.layers.Softmax(axis=-2)
    self.noise = noiseLayer(0.00001,0.00009)
  def call(self, input_tensor, training=False):
    x = self.layer1(input_tensor,training=training)
    x = self.layer2(x,training=training)
    x = self.layerL1(x,training=training)
    x = self.layerL2(x,training=training)
    x = self.noise(x)
    return x

class ConditionalNN12(tf.keras.Model):
  def __init__(self, out_dim, l1 = 0, l2 = 0):
    super(ConditionalNN12, self).__init__(name='ConditionalNN12')
    self.layer1 = tf.keras.layers.Dense(20,'tanh',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer2 = tf.keras.layers.Dense(20,'tanh',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL1 = tf.keras.layers.Dense(1,'linear',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL2 = tf.keras.layers.Softmax(axis=-2)
    self.noise = noiseLayer(0.00001,0.00009)
  def call(self, input_tensor, training=False):
    x = self.layer1(input_tensor,training=training)
    x = self.layer2(x,training=training)
    x = self.layerL1(x,training=training)
    x = self.layerL2(x,training=training)
    x = self.noise(x)
    return x

class ConditionalNN21(tf.keras.Model):
  def __init__(self, out_dim, l1 = 0, l2 = 0):
    super(ConditionalNN21, self).__init__(name='ConditionalNN21')
    self.layer1 = tf.keras.layers.Dense(10,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer2 = tf.keras.layers.Dense(10,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer3 = tf.keras.layers.Dense(10,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer4 = tf.keras.layers.Dense(10,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL1 = tf.keras.layers.Dense(1,'linear',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL2 = tf.keras.layers.Softmax(axis=-2)
    self.noise = noiseLayer(0.00001,0.00009)
  def call(self, input_tensor, training=False):
    x = self.layer1(input_tensor,training=training)
    x = self.layer2(x,training=training)
    x = self.layer3(x,training=training)
    x = self.layer4(x,training=training)
    x = self.layerL1(x,training=training)
    x = self.layerL2(x,training=training)
    x = self.noise(x)
    return x

class ConditionalNN22(tf.keras.Model):
  def __init__(self, out_dim, l1 = 0, l2 = 0):
    super(ConditionalNN22, self).__init__(name='ConditionalNN22')
    self.layer1 = tf.keras.layers.Dense(20,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer2 = tf.keras.layers.Dense(20,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer3 = tf.keras.layers.Dense(20,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer4 = tf.keras.layers.Dense(20,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL1 = tf.keras.layers.Dense(1,'linear',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL2 = tf.keras.layers.Softmax(axis=-2)
    self.noise = noiseLayer(0.00001,0.00009)
  def call(self, input_tensor, training=False):
    x = self.layer1(input_tensor,training=training)
    x = self.layer2(x,training=training)
    x = self.layer3(x,training=training)
    x = self.layer4(x,training=training)
    x = self.layerL1(x,training=training)
    x = self.layerL2(x,training=training)
    x = self.noise(x)
    return x

class DeepConditionalNN(tf.keras.Model):
  def __init__(self, out_dim, l1 = 0, l2 = 0):
    super(DeepConditionalNN, self).__init__(name='DeepConditionalNN')
    self.layer1 = tf.keras.layers.Dense(20,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer2 = tf.keras.layers.Dense(20,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer3 = tf.keras.layers.Dense(20,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer4 = tf.keras.layers.Dense(20,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer5 = tf.keras.layers.Dense(20,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer6 = tf.keras.layers.Dense(20,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layer7 = tf.keras.layers.Dense(20,'relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL1 = tf.keras.layers.Dense(1,'linear',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2))
    self.layerL2 = tf.keras.layers.Softmax(axis=-2)
    self.noise = noiseLayer(0.00001,0.00009)
  def call(self, input_tensor, training=False):
    x = self.layer1(input_tensor,training=training)
    x = self.layer2(x,training=training)
    x = self.layer3(x,training=training)
    x = self.layer4(x,training=training)
    x = self.layer5(x,training=training)
    x = self.layer6(x,training=training)
    x = self.layer7(x,training=training)
    x = self.layerL1(x,training=training)
    x = self.layerL2(x,training=training)
    x = self.noise(x)
    return x

import matplotlib.pyplot as plt

gc.collect()

# Utility: measures distance between two 2D coordinates
def dist(a,b):
    return np.sqrt(np.square(a[0]-b[0]) + np.square(a[1]-b[1]))

# Parameter choices for testing
learning_rates = [0.00001,0.0001,0.001,0.005]
l1s = [0.005,0.00005,0]
l2s = [0.005,0.00005,0]
mini_batch_sizes = [512,1024]

# Logging:
log = {}

all_models =    [ConditionalLogisticRegression(out_dim = 201, l1 = 0, l2 = 0),
                 ConditionalNN1(out_dim = 201,    l1 = 0, l2 = 0),
                 ConditionalNN2(out_dim = 201,    l1 = 0, l2 = 0),
                 ConditionalNN11(out_dim = 201,   l1 = 0, l2 = 0),
                 ConditionalNN12(out_dim = 201,   l1 = 0, l2 = 0),
                 ConditionalNN21(out_dim = 201,   l1 = 0, l2 = 0),
                 ConditionalNN22(out_dim = 201,   l1 = 0, l2 = 0),
                 DeepConditionalNN(out_dim = 201, l1 = 0, l2 = 0)]

print("\nStarting now\n")

# Test N configurations
for n in range(0,25,1):
    
    np.random.seed(n*n*2)
    tf.random.set_seed(n*n)
    
    print("\n============================================\n\n"+str(n)+"-th parameter configuration:\n")
    
    #Sample hyperparameter configuration
    learning_rate = np.random.choice(learning_rates)
    l1 = np.random.choice(l1s)
    l2 = np.random.choice(l2s)
    mini_batch_size = np.random.choice(mini_batch_sizes)
    
    print("Learning Rate:",learning_rate)
    print("L1:",l1)
    print("L2:",l2)
    print("Minibatch Size:",mini_batch_size)
    print()
    
    # Log n-th configuration
    log['parameter_config'+str(n)] = {'learning_rate':learning_rate,
                                      'l1':l1,
                                      'l2':l2}
    # Test every model
    for model in all_models:
        
        print("\n    Model:",model.name)
        
        # log model being tested
        log['parameter_config'+str(n)]['model_'+model.name] = {'model_name':model.name}
    
        # Folding
        for L in range(n_folds):
            
            print("\n        "+str(L)+"-th fold\n")

            x_train = tf.concat(x_folds[:L] + x_folds[L+1:], axis=0)
            x_test = tf.concat(x_folds[L],axis=0)

            y_train = tf.concat(y_folds[:L] + y_folds[L+1:], axis=0)
            y_test = y_folds[L]
            
            log['parameter_config'+str(n)]['minibatch_size'] = mini_batch_size
            
            model.__init__(out_dim=201,l1=l1,l2=l2)
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss="categorical_crossentropy",metrics=[tf.keras.metrics.CategoricalAccuracy()])
            t = time.time()
            history = model.fit(x_train[:,:,0:40], y_train, epochs = 750,
                      batch_size = mini_batch_size, shuffle = False, verbose = 0, validation_split=0.05,
                      callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=35)])
            t = time.time()-t

            # Evaluate model on training and test sets
            loss_train, acc_train = model.evaluate(x_train[:,:,0:40],y_train,batch_size=x_train.shape[0])
            loss_test, acc_test = model.evaluate(x_test[:,:,0:40],y_test,batch_size=x_test.shape[0])

            # Log L-th fold training/test info
            log['parameter_config'+str(n)]['model_'+model.name]['fold_'+str(L)] = {
                'training_loss':loss_train,
                'training_accuracy':acc_train,
                'testing_loss':loss_test,
                'testing_accuracy':acc_test,
                'elapsed_time':t,
                'train_set_size':x_train.shape[0],
                'test_set_size':x_test.shape[0],
                'epochs':len(history.history['loss'])
            }

            sum__ = 0
            sum2__ = 0
            for i in range(n_folds):
                if i == L:
                    continue

                coords_tensor =  tf.concat([x_folds[i],tf.cast(tf.reshape(y_folds[i], shape = (y_folds[i].shape[0],y_folds[i].shape[1],1)),'float64')],axis=2)
                true_coords = tf.reshape(coords_tensor, shape = (coords_tensor.shape[0]*coords_tensor.shape[1],coords_tensor.shape[2]))
                true_coords = tf.gather(true_coords,tf.where(true_coords[:,-1] == 1))[:,:,-3:-1]
                true_coords = tf.reshape(true_coords, shape = (true_coords.shape[0],-1))
                
                pred = to_categorical(np.argmax(model.predict(x_folds[i][:,:,0:40]), axis=1), 201)
                pred = tf.concat([x_folds[i],tf.cast(tf.reshape(pred, shape = (pred.shape[0],pred.shape[1],1)),'float64')],axis=2)
                pred_coords = tf.reshape(pred, shape = (pred.shape[0]*pred.shape[1],pred.shape[2]))
                pred_coords = tf.gather(pred_coords,tf.where(pred_coords[:,-1] == 1))[:,:,-3:-1]
                pred_coords = tf.reshape(pred_coords, shape = (pred_coords.shape[0],-1))

                sum_ = 0
                for j in range(len(pred_coords)):
                    sum_ += dist(pred_coords.numpy()[j], true_coords.numpy()[j])
                sum__ += sum_
                sum_ /= len(pred_coords)

                # Log model being trained information
                log['parameter_config'+str(n)]['model_'+model.name]['fold_'+str(L)]["fold_"+str(i)+"_mean_distance_error"] = sum_

                Z = x_folds[i]
                Z = Z.numpy()
                Z[:,-1,:] = np.mean(Z,axis=1)
                Z = tf.convert_to_tensor(Z)
                
                pred = to_categorical(np.argmax(model.predict(Z[:,:,0:40]), axis=1), 201)
                pred = tf.concat([Z,tf.cast(tf.reshape(pred, shape = (pred.shape[0],pred.shape[1],1)),'float64')],axis=2)
                pred_coords = tf.reshape(pred, shape = (pred.shape[0]*pred.shape[1],pred.shape[2]))
                pred_coords = tf.gather(pred_coords,tf.where(pred_coords[:,-1] == 1))[:,:,-3:-1]
                pred_coords = tf.reshape(pred_coords, shape = (pred_coords.shape[0],-1))

                sum_ = 0
                for j in range(len(pred_coords)):
                    sum_ += dist(pred_coords.numpy()[j], true_coords.numpy()[j])
                sum2__ += sum_
                sum_ /= len(pred_coords)

                # Log model being trained information
                log['parameter_config'+str(n)]['model_'+model.name]['fold_'+str(L)]["fold_"+str(i)+"_mean_distance_error_2"] = sum_

            sum__ /= x_train.shape[0]
            sum2__ /= x_train.shape[0]
            # Log model being trained information
            log['parameter_config'+str(n)]['model_'+model.name]['fold_'+str(L)]["mean_distance_error_training_jaguars"] = sum__
            log['parameter_config'+str(n)]['model_'+model.name]['fold_'+str(L)]["mean_distance_error_training_jaguars_2"] = sum2__
            
            # Evaluate for left out fold
            coords_tensor =  tf.concat([x_folds[L],tf.cast(tf.reshape(y_folds[L], shape = (y_folds[L].shape[0],y_folds[L].shape[1],1)),'float64')],axis=2)
            true_coords = tf.reshape(coords_tensor, shape = (coords_tensor.shape[0]*coords_tensor.shape[1],coords_tensor.shape[2]))
            true_coords = tf.gather(true_coords,tf.where(true_coords[:,-1] == 1))[:,:,-3:-1]
            true_coords = tf.reshape(true_coords, shape = (true_coords.shape[0],-1))
                
            pred = to_categorical(np.argmax(model.predict(x_folds[L][:,:,0:40]), axis=1), 201)
            pred = tf.concat([x_folds[L],tf.cast(tf.reshape(pred, shape = (pred.shape[0],pred.shape[1],1)),'float64')],axis=2)
            pred_coords = tf.reshape(pred, shape = (pred.shape[0]*pred.shape[1],pred.shape[2]))
            pred_coords = tf.gather(pred_coords,tf.where(pred_coords[:,-1] == 1))[:,:,-3:-1]
            pred_coords = tf.reshape(pred_coords, shape = (pred_coords.shape[0],-1))
            
            sum_ = 0
            for j in range(len(pred_coords)):
                sum_ += dist(pred_coords.numpy()[j], true_coords.numpy()[j])
            sum_ /= len(pred_coords)

            # Log model on test data
            log['parameter_config'+str(n)]['model_'+model.name]['fold_'+str(L)]["fold_"+str(L)+"_mean_distance_error"] = sum_

            Z = x_folds[L]
            Z = Z.numpy()
            Z[:,-1,:] = np.mean(Z,axis=1)
            Z = tf.convert_to_tensor(Z)

            pred = to_categorical(np.argmax(model.predict(Z[:,:,0:40]), axis=1), 201)
            pred = tf.concat([Z,tf.cast(tf.reshape(pred, shape = (pred.shape[0],pred.shape[1],1)),'float64')],axis=2)
            pred_coords = tf.reshape(pred, shape = (pred.shape[0]*pred.shape[1],pred.shape[2]))
            pred_coords = tf.gather(pred_coords,tf.where(pred_coords[:,-1] == 1))[:,:,-3:-1]
            pred_coords = tf.reshape(pred_coords, shape = (pred_coords.shape[0],-1))
            
            sum_ = 0
            for j in range(len(pred_coords)):
                sum_ += dist(pred_coords.numpy()[j], true_coords.numpy()[j])
            sum_ /= len(pred_coords)

            # Log model on test data
            log['parameter_config'+str(n)]['model_'+model.name]['fold_'+str(L)]["fold_"+str(L)+"_mean_distance_error_2"] = sum_
            
            # summarize history for accuracy
            plt.plot(history.history['categorical_accuracy'])
            plt.plot(history.history['val_categorical_accuracy'])
            plt.title('model accuracy')
            plt.ylabel('accuracy')
            plt.xlabel('epoch')
            plt.legend(['train', 'test'], loc='upper left')
            plt.savefig('./log3/accuracy_config_'+str(n)+'_'+model.name+'_fold_'+str(L))
            plt.clf()
            
            # summarize history for loss
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('model loss')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            plt.legend(['train', 'test'], loc='upper left')
            plt.savefig('./log3/loss_config_'+str(n)+'_'+model.name+'_fold_'+str(L))
            plt.clf()

            gc.collect()

        # Temporarily save log files
        pickle.dump(log,open("/content/drive/MyDrive/Trabalho de Formatura/00 - Animal Movement/animal-movement-model/log3/log0"+".pkl",'wb'))
        gc.collect()
        
    # Temporarily save log files
    pickle.dump(log,open("/content/drive/MyDrive/Trabalho de Formatura/00 - Animal Movement/animal-movement-model/log3/log0"+".pkl",'wb'))
    gc.collect()  

# Definitively save log files
pickle.dump(log,open("/content/drive/MyDrive/Trabalho de Formatura/00 - Animal Movement/animal-movement-model/log3/log0"+".pkl",'wb'))
gc.collect() 



Packages imported successfully



Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/lo


Starting now





Model: ConditionalLogisticRegression

Config: [0.005, 0, 5e-05, 512]

        0-th fold

1/1 [==============================] - 0s 184ms/step - loss: 5.1010 - categorical_accuracy: 0.0382

        1-th fold

1/1 [==============================] - 0s 209ms/step - loss: 5.0549 - categorical_accuracy: 0.0410

        2-th fold

1/1 [==============================] - 0s 191ms/step - loss: 5.0405 - categorical_accuracy: 0.0468

        3-th fold

1/1 [==============================] - 0s 193ms/step - loss: 5.0889 - categorical_accuracy: 0.0396

        4-th fold

1/1 [==============================] - 0s 180ms/step - loss: 5.1041 - categorical_accuracy: 0.0310

        5-th fold

1/1 [==============================] - 0s 188ms/step - loss: 5.0740 - categorical_accuracy: 0.0310

        6-th fold

1/1 [==============================] - 0s 195ms/step - loss: 5.0427 - categorical_accuracy: 0.0331




Model: ConditionalLogisticRegression

Config: [0.001, 0, 0, 1024]

        

0

<Figure size 432x288 with 0 Axes>

In [ ]:
while True:
  pass